# 06. テンプレートマッチング
講義で説明する画像処理の方法について，google colaboratoryを利用して演習する．
google colaboratoryは，クラウドで実行する Jupyter ノートブック環境である.
google coraboratoryについては，[ここ](https://www.tdi.co.jp/miso/google-colaboratory-gpu)や[ここ](https://www.codexa.net/how-to-use-google-colaboratory/)を参考にすること．

下記のプログラムを実行すると，様々な類似尺度を用いたテンプレートマッチングや高速化のための粗密探索を実行する．

## 準備
プログラムの動作に必要なデータをダウンロードし，zipファイルを解凍する．
`!`で始まるコマンドはPythonではなく，Linux（Ubuntu）のコマンドを実行している．

In [ ]:
!wget -q http://www.mprg.cs.chubu.ac.jp/Tutorial/ML_Lecture/tutorial_ip_2020/image1.zip
!unzip -q image1.zip
!ls
!ls ./image1/

## 画像の読み込みと表示
必要なパッケージをインポートし，画像を表示する．

In [ ]:
import math
import cv2
import numpy as np
from matplotlib import pyplot as plt

img1 = cv2.imread('./image1/woman-t.jpg', 0)
img2 = cv2.imread('./image1/woman-g.jpg', 0)

plt.imshow(img1, cmap="gray", clim=(0, 255))
plt.title("template")
plt.show()

plt.imshow(img2, cmap="gray", clim=(0, 255))
plt.title("original")
plt.show()

## 類似尺度

テンプレートマッチングでは，テンプレートと入力画像の画素比較によって類似度を算出する．類似度の高い位置を算出することで入力画像中のテンプレート画像位置を探索できる．

テンプレートマッチングで使用する類似尺度は様々なものがある．
ここでは，以下の類似尺度を使用してテンプレートマッチングを行う．

* Sum of Squared Difference (SSD)
* Sum of Absolute Difference (SAD)
* 正規化相互相関（Normalized Cross-Correlation）
* ゼロ平均相互相関係数（Zero-means Normalized Cross-Correlation）

まず，はじめに類似尺度を計算するための関数を定義する．
ここでは，テンプレートと類似度を計算したい画像の局所領域を入力した際に，その類似度を計算して返すような関数を定義する．

全ての関数において，入力された画像は`flatten`関数によって2次元配列から1次元配列へと並び替えられる（`ravel`と同様の効果）．

その後，各尺度の定義に合わせて類似度を計算する．
ここでは，Numpy配列の演算を利用して類似度計算を行っている．
Numpy配列の演算では，配列の各要素に対して同一の演算を行うことが可能となる．
例えば，`img1 - img2`では，`img1`の各要素と`img2`の各要素の減算を行なっており，`diff**2`では各要素の2乗を計算している．
Numpy配列の演算を用いることで，これまでに行なってきたfor文で一つづつ計算するよりも高速に計算することが可能となる．

In [ ]:
def ssd(img1, img2):
  img1 = np.float32(img1.flatten())
  img2 = np.float32(img2.flatten())

  diff = img1 - img2
  ssd_val = np.sum(diff**2)
  return ssd_val

  
def sad(img1, img2):
  img1 = np.float32(img1.flatten())
  img2 = np.float32(img2.flatten())
  
  sad_val = np.sum(np.abs(img1 - img2))
  return sad_val


def norm_cross_correlation(img1, img2):
  img1 = np.float32(img1.flatten())
  img2 = np.float32(img2.flatten())
  
  sum1 = np.sum(img1**2)
  sum2 = np.sum(img2**2)
  sum_prod = np.sum(img1 * img2)
  ncc_val = sum_prod / math.sqrt(sum1 * sum2)
  return ncc_val


def zeromean_norm_cross_correlation(img1, img2):
  img1 = np.float32(img1.flatten())
  img2 = np.float32(img2.flatten())

  mean1 = np.mean(img1)
  mean2 = np.mean(img2)

  subtract_mean1 = img1 - mean1
  subtract_mean2 = img2 - mean2

  sum1 = np.sum(subtract_mean1**2)
  sum2 = np.sum(subtract_mean2**2)

  sum_prod = np.sum(subtract_mean1 * subtract_mean2)

  zm_ncc_val = sum_prod / math.sqrt(sum1 * sum2)
  return zm_ncc_val

次に，指定した画像とテンプレートでテンプレートマッチングを行うための関数`matching_template`を定義する．

この関数では，探索したい元画像とテンプレートを引数として入力する．
また，第3引数の`metric`は，使用する類似尺度を指定する．
`matching_template`関数では，上で定義した類似尺度の**関数を引数として入力**する．これにより関数中の`metric(...)`において，引数で指定した類似尺度を計算するための関数を用いて類似度計算を行うことができる．
第4引数はもっとも類似度が高い座標を求めるために，最小値（最大値）を探すかどうかを指定する引数である．
類似尺度によって，最も低い（最も高い）箇所が探索結果となる．
そのため，第3引数で指定した類似尺度に合わせて最小値または最大値の座標のどちらを求めるかを指定する．
`find_min=True`となっているのは，デフォルト引数であり．関数を使用する際に，この引数を記述しない場合は，`True`がデフォルト値として使用されることを示している．

画像，テンプレート，尺度等を指定したのち，テンプレートマッチングを行う．

まず，類似度を保存するための配列`template_socre`を用意する．
その後，for文で1領域ごとに類似度を計算し，その結果を保存する．
類似度の計算が全ての領域で終わると，`find_min`に従って，類似度が最小値または最大値の座標を求める．
`argmin`関数は要素の値が最小となる箇所のインデックスを計算する関数であるが，通常は1次元配列に対して使用するものであり．2次元配列に対するインデックスを計算することができない．
ここでは`unravel_index`関数を用いることで，2次元配列のまま最小値（最大値）のインデックスを計算している．

得られた座標は左上の座標であり，領域を指定するため，右下の座標をテンプレートサイズから計算し，

1. 探索した類似度のマップ
2. マッチング結果の左上座標
3. マッチング結果の右下座標

を返す．

In [ ]:
def matching_template(img, template, metric, find_min=True):
  h, w = img.shape
  ht, wt = template.shape
  template_score = np.zeros([h - ht, w - wt], dtype=np.float32)

  for y in range(h - ht):
    for x in range(w - wt):
      template_score[y, x] = metric(img[y:y+ht, x:x+wt], template)
  
  if find_min:
    top_left = np.unravel_index(np.argmin(template_score), template_score.shape)
  else:
    top_left = np.unravel_index(np.argmax(template_score), template_score.shape)

  bottom_right = (top_left[0] + ht, top_left[1] + wt)
  return template_score, top_left[::-1], bottom_right[::-1]

### Sum of Squared Difference (SSD)

SSDを類似尺度として用いた場合のテンプレートマッチングを行う．
テンプレートマッチングの処理は，すでに上の関数で全て定義してあるため．`matching_template`関数の第3引数を`ssd`と指定することで計算が可能である．
また，SSDは値が小さい場合に類似度が高くなるため，第4引数の`find_min`をTrueにする必要があるが，デフォルト値ですでに`True`が指定されているため，記述していない．

マッチング結果が得られたら，`rectangle`関数を用いて矩形を描画し，結果を表示する．



In [ ]:
img = img2.copy()
template = img1.copy()

ssd_score, top_left_coord, bottom_right_coord = matching_template(img, template, ssd)

print("found coordinate (x, y)")
print("   top-left    :", top_left_coord)
print("   bottom-right:", bottom_right_coord)

cv2.rectangle(img, top_left_coord, bottom_right_coord, 255, 5)

plt.figure(figsize=(12, 4))
plt.subplot(121), plt.imshow(ssd_score, cmap="jet")
plt.colorbar()
plt.subplot(122), plt.imshow(img, cmap="gray", clim=(0, 255))
plt.show()

### Sum of Absolute Difference (SAD)

SADもSDDと同様の手順でテンプレートマッチングを行う．
そのため，`matching_template`関数の第3引数を`sad`に変更するだけで異なる類似尺度での探索が可能である．



In [ ]:
img = img2.copy()
template = img1.copy()

sad_score, top_left_coord, bottom_right_coord = matching_template(img, template, sad)

print("found coordinate (x, y)")
print("   top-left    :", top_left_coord)
print("   bottom-right:", bottom_right_coord)

cv2.rectangle(img, top_left_coord, bottom_right_coord, 255, 5)

plt.figure(figsize=(12, 4))
plt.subplot(121), plt.imshow(sad_score, cmap="jet")
plt.colorbar()
plt.subplot(122), plt.imshow(img, cmap="gray", clim=(0, 255))
plt.show()

### 正規化相互相関（Normalized Cross-Correlation）

正規化相互相関も，SSDおよびSADと同様の手順で実行することが可能であるが，
この類似尺度では最も値が高い箇所をマッチング結果として探索する必要があるため，第4引数を`find_min=False`と指定して，最大値を探すようにする必要がある．


In [ ]:
img = img2.copy()
template = img1.copy()

ncc_score, top_left_coord, bottom_right_coord = matching_template(img, template, norm_cross_correlation, find_min=False)

print("found coordinate (x, y)")
print("   top-left    :", top_left_coord)
print("   bottom-right:", bottom_right_coord)

cv2.rectangle(img, top_left_coord, bottom_right_coord, 255, 5)

plt.figure(figsize=(12, 4))
plt.subplot(121), plt.imshow(ncc_score, cmap="jet")
plt.colorbar()
plt.subplot(122), plt.imshow(img, cmap="gray", clim=(0, 255))
plt.show()

### ゼロ平均相互相関係数（Zero-means Normalized Cross-Correlation）

ゼロ平均相互相関係数についても，正規化相互相関と同様の手順で実行が可能である．

ゼロ平均相互相関係数の尺度を用いた場合．正規化相互相関よりもマッチングする箇所とそうでない箇所の類似度の差が大きいことが確認できる．



In [ ]:
img = img2.copy()
template = img1.copy()

zm_ncc_score, top_left_coord, bottom_right_coord = matching_template(img, template, zeromean_norm_cross_correlation, find_min=False)

print("found coordinate (x, y)")
print("   top-left    :", top_left_coord)
print("   bottom-right:", bottom_right_coord)

cv2.rectangle(img, top_left_coord, bottom_right_coord, 255, 5)

plt.figure(figsize=(12, 4))
plt.subplot(121), plt.imshow(zm_ncc_score, cmap="jet")
plt.colorbar()
plt.subplot(122), plt.imshow(img, cmap="gray", clim=(0, 255))
plt.show()

## 粗密探索

テンプレートマッチングは探索する画像の大きさ等に応じて処理時間が増加する．
そのため，大きな画像に対してマッチングを行う場合には，効率的な探索を行う必要がある．

効率的なテンプレートマッチングの方法として粗密探索がある．
粗密探索では，はじめに，低解像度の小さな画像で探索を行う．
その結果得られた座標の近傍のみで，高解像度な探索を行うことで効率化を行う方法である．

まずはじめに，従来の全探索（全ての領域で類似度計算を行う方法）の処理時間を計測する．
時間の計測には`time`モジュールの`time`関数（`time.time()`）使用する．
`time`関数は，この関数が実行された際の時刻を保存する関数である．
そのため，テンプレートマッチングの開始前後の時刻を記録して，その差分を計算することで処理時間を計測することが可能となる．

テンプレートマッチングの実行には，4つの類似尺度のうち，最も計算時間がかかると思われる，ゼロ平均相互相関係数を使用する．


In [ ]:
import time

img = img2.copy()
template = img1.copy()

time_start1 = time.time()
zm_ncc_score, top_left_coord, bottom_right_coord = matching_template(img, template, zeromean_norm_cross_correlation, find_min=False)
time_end1 = time.time()
print("found top-left coordinate:", top_left_coord)
print("processing time:", time_end1 - time_start1, "[s]")

次に粗密探索によるテンプレートマッチングを実行し，処理時間を計測する．

はじめに，前準備として，低解像度の画像とテンプレート（`img_small`, `template_small`）を用意する．
ここでは，元のサイズの1/4のサイズにリサイズする．

`time`関数で時間を計測し，低解像度画像でテンプレートマッチングを行う．
その後，低解像度画像で得られた結果に対応する領域を元画像から抽出する．
そして，対応領域のみを用いてテンプレートマッチングを行い，得られた結果を元画像の座標へと変換してマッチングを終了する．

実行の結果，上で実行した全探索に比べて大幅に処理時間を削減できていることがわかる．


In [ ]:
img = img2.copy()
template = img1.copy()
ht, wt = template.shape

# 画像を1/4のサイズにリサイズ
img_small = cv2.resize(img, None, fx=0.25, fy=0.25)
template_small = cv2.resize(template, None, fx=0.25, fy=0.25)

time_start2 = time.time()
# 低解像度画像でテンプレートマッチング
score_s, tl_s, br_s = matching_template(img_small, template_small, zeromean_norm_cross_correlation, find_min=False)
print("found top-left coordinate (small)   :", tl_s)

# 低解像度画像に対応する領域を元画像から抽出
img_trimmed = img[tl_s[0]*4:(tl_s[0]+1)*4+ht, tl_s[1]*4:(tl_s[1]+1)*4+wt]

# 対応領域を抽出した画像でテンプレートマッチング
score, tl, br = matching_template(img_trimmed, template, zeromean_norm_cross_correlation, find_min=False)
print("found top-left coordinate (trimmed) :", tl)

# 得られた座標を元画像中の座標に変換
top_left = (tl[0] + tl_s[0]*4, tl[1] + tl_s[1]*4)
bottom_right = (br[0] + tl_s[0]*4, br[1] + tl_s[1]*4)
print("found top-left coordinate (original):", top_left)
time_end2 = time.time()
print("processing time:", time_end2 - time_start2, "[s]")

# 結果の描画
cv2.rectangle(img_small, tl_s, br_s, 255, 1)
cv2.rectangle(img, top_left, bottom_right, 255, 5)

# 表示
plt.figure(figsize=(12, 4))
plt.subplot(121), plt.imshow(score_s, cmap="jet")
plt.colorbar()
plt.subplot(122), plt.imshow(img_small, cmap="gray", clim=(0, 255))
plt.show()

plt.figure(figsize=(12, 4))
plt.subplot(121), plt.imshow(score, cmap="jet")
plt.colorbar()
plt.subplot(122), plt.imshow(img, cmap="gray", clim=(0, 255))
plt.show()

## OpenCVを用いた方法

OpenCVにはテンプレートマッチングを行うための，`matchTemplate`関数がある．
この関数では，いくつかの類似尺度を使用することができる．

以下では，`matchTemplate`関数を用いて，それぞれの類似尺度を用いた場合の結果を示す．
なお，それぞれの類似尺度の詳細については[OpenCVのドキュメント](https://docs.opencv.org/master/df/dfb/group__imgproc__object.html)を参照すること．

In [ ]:
template = img1.copy()
h, w = template.shape

# 6つの方法で比較を行う
methods = ['cv2.TM_SQDIFF',         # 差分相関 (SSD)
           'cv2.TM_SQDIFF_NORMED',  # 正規化差分相関
           'cv2.TM_CCORR',          # 相互相関
           'cv2.TM_CCORR_NORMED',   # 正規化相互相関
           'cv2.TM_CCOEFF',         # 相関係数
           'cv2.TM_CCOEFF_NORMED']  # 正規化相関係数

# 上で定義した方法を一つづつ実行
for meth in methods:
  img = img2.copy()
  method = eval(meth)

  # テンプレートマッチング
  res = cv2.matchTemplate(img, template, method)
  min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

  # メソッドが TM_SQDIFF または TM_SQDIFF_NORMED の場合は，最小値を取る
  if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
    top_left = min_loc
  else:
    top_left = max_loc
  bottom_right = (top_left[0] + w, top_left[1] + h)

  cv2.rectangle(img,top_left, bottom_right, 255, 5)

  plt.figure(figsize=(12, 4))
  plt.subplot(121), plt.imshow(res, cmap='jet')
  plt.title('Matching Score')
  plt.colorbar()
  plt.subplot(122), plt.imshow(img, cmap='gray', clim=(0, 255))
  plt.title('Detected Point')
  plt.suptitle(meth)

  plt.show()

## チャンファーマッチング (Chamfer Matching)

チャンファーマッチング (Chamfer Matching) は，入力画像とテンプレート画像間のエッジの相違度に基づいて探索を行う手法である．

チャンファーマッチングでは，まず画像のエッジを抽出し，エッジ画像に距離変換処理を行い距離変換画像を作成する．この距離変換画像ではエッジの距離を0とし，エッジから離れるほど距離が増加する．入力画像から作成した距離変換画像とテンプレート画像から作成したエッジ画像を相違度に基づいていマッチングを行う．

まずはじめに，入力画像とテンプレート画像のエッジ画像を作成する．エッジ検出法として，OpenCVの関数として用意されているCannyのエッジ検出器を使用する．
この時，検出されたエッジ画素は255の値を保持しているが，後の処理のために，テンプレート画像のエッジの値を255から1に置き換える．

In [ ]:
# 使用する画像とテンプレートを用意
img = img2.copy()
template = img1.copy()

# Cannyのエッジ検出器を画像に適用してエッジ検出
img_edge = cv2.Canny(img, 150, 500)
template_edge = cv2.Canny(template, 150, 500)
template_edge[template_edge == 255] = 1 # エッジの画素の値を255 --> 1に変換

# エッジの検出結果を表示
plt.figure(figsize=(12, 4))
plt.subplot(121), plt.imshow(template_edge, cmap='gray', clim=(0, 1))
plt.title('Detected Edge of Template')
plt.subplot(122), plt.imshow(img_edge, cmap='gray', clim=(0, 255))
plt.title('Detected Edge of Input Image')
plt.show()

次に入力画像における距離変換画像を作成する．

ここでは，距離変換画像の作成方法として，`cv2.distanceTransform`を使用する．
このとき，`cv2.distanceTransform`の計算では，画素値が0の画素からの距離を算出するため，
エッジ画像の画素値を反転させて距離変換画像の作成を行うことに注意されたい．


In [ ]:
# 距離変換画像の作成
img_dist = cv2.distanceTransform(255 - img_edge, cv2.DIST_L1, 0)

plt.figure()
plt.imshow(img_dist, cmap='gray')
plt.colorbar()
plt.show()

上記のテンプレート画像のエッジ画像と距離変換画像を用いてチャンファーマッチングによる探索を行う．

まず，手動で設定するパラメータとして，チャンファーマッチングにおける更新量のパラメータ`alpha`と初期探索点`(u, v)`を設定する．

その後，距離変換画像におけるx, y方向それぞれの勾配を算出する．勾配の算出には，「02. 空間フィルタリング」の微分フィルタを用いる．
さらに，テンプレート画像のエッジ画像におけるエッジの点数`n`を算出し，探索途中における探索点を保存するためのリスト`result`を作成する．

チャンファーマッチングによる探索では，
まずはじめに，距離変換画像とその勾配のうち，現在の探索点`(u, v)`に対応する領域の情報を取得する．
次に，相違度`s`の算出を行い，相違度に対する勾配`ds_dx`および`ds_dy`を算出する．
算出した勾配を用いて，現在の探索点を更新する．
この時，勾配の大きさおよび相違度が閾値以下となった場合に探索を終了する．
この条件を満たすまで探索を繰り返すことでマッチングを行う．



In [ ]:
ht, wt = template_edge.shape

# 更新量
alpha = 5.0

# 初期探索点 (u, v) を指定
u, v = 10, 10  # x, y方向

# 距離変換画像の勾配算出
img_grad_x = img_dist[:, 1:] - img_dist[:, :-1]
img_grad_y = img_dist[1:, :] - img_dist[:-1, :]

# エッジの点数
n = np.sum(template_edge)

# 探索途中の結果を保存するためのリストを作成
result = [(u, v)]

# 探索開始前の座標（初期値）を表示
print("initial coordinate: (%d, %d)" % (u, v))

# 探索を行うための無限ループ（終了条件は一番下の部分で定義）
while True:
  img_dist_part = img_dist[v:v+ht, u:u+wt]
  grad_x_part = img_grad_x[v:v+ht, u:u+wt]
  grad_y_part = img_grad_y[v:v+ht, u:u+wt]

  # 相違度sの算出
  s = np.sum(template_edge * img_dist_part) / n

  # 相違度sの勾配の算出
  ds_dx = np.sum(template_edge * grad_x_part) / n
  ds_dy = np.sum(template_edge * grad_y_part) / n

  # 勾配の大きさと角度を算出
  grad_magnitude = math.sqrt(ds_dx**2 + ds_dy**2)
  grad_orientation = math.degrees(math.atan(ds_dy / ds_dx))

  # 更新
  u = u - round(alpha * ds_dx)
  v = v - round(alpha * ds_dy)

  # 途中結果の保存
  result.append((u, v))

  # 情報の表示
  print("u, v: %d, %d" % (u, v))
  print("   dx, dy: %0.2f, %0.2f (magnitude: %0.2f, orientation: %0.2f)" % (ds_dx, ds_dy, grad_magnitude, grad_orientation))
  print("   s:", s, "\n")

  # 勾配の大きさおよび相違度が閾値以下になった場合に探索を終了する
  if grad_magnitude < 0.4 and s < 1.0:
    break

最後に探索した結果を描画する．

上の探索中に保存した探索点とそれに対応する範囲の矩形を描画する．
ここでは，10回に1回の割合で途中の探索点を描画する．
最後に探索結果を白の矩形で描画し，表示を行う．

In [ ]:
# 結果を描画するための画像データをコピーして用意
detected = img2.copy()

# 探索途中の結果を描画（10回に1回）（グレー）
for i in range(0, len(result), 10):
  top_left = result[i]
  bottom_right = (top_left[0]+wt, top_left[1]+ht)
  detected = cv2.rectangle(detected, top_left, bottom_right, 127, 5)

# 探索結果（一番最後の探索）を描画（白）
top_left = result[-1]
bottom_right = (top_left[0]+wt, top_left[1]+ht)
detected = cv2.rectangle(detected, top_left, bottom_right, 255, 5)

# 描画結果の表示
plt.figure()
plt.imshow(detected, cmap='gray', clim=(0, 255))
plt.show()

## 課題

* ヒートマップを観察して，考察すること
* 各手法の処理時間を比較し，考察すること
* 粗密探索において，低解像度画像のサイズを変更し，処理時間の違いを確認すること